# 본 코드는 3_0 코드를 돌린 이후에 입력 데이터를 time-normalized 할 때 사용!
# 필터가 적용된 데이터를 불러온다!

In [3]:
# 필요한 library
import os
from re import A
import subprocess
from natsort import natsorted
import pandas as pd
import keyboard
import signal
import subprocess
import psutil
import math
import shutil
import argparse
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

In [4]:
# 필요한 디렉토리 만들기
# 폴더 있어도 새로 만들기
def refresh_dir(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)
    os.makedirs(file_path)
# 폴더 없으면 
def make_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)
# 데이터의 모든 columns을 101개로 만들기!
def resample_ALL(datum):
    newColumns = datum.columns
    # filter 처리한 새로운 dataframe 만들기
    norm_datum = pd.DataFrame(columns=newColumns)
    for i in range(0,len(datum.columns)): # IMU 데이터만 filtering 하기
        raw_Data = datum.iloc[:,i]
        norm_Data = signal.resample(raw_Data,101) # 101개로 모든 데이터 수 일치시키기
        norm_datum[datum.columns[i]] = norm_Data
    return norm_datum
    
def resample_poly_ALL(datum):
    newColumns = datum.columns
    # filter 처리한 새로운 dataframe 만들기
    norm_datum = pd.DataFrame(columns=newColumns)
    for i in range(0,len(datum.columns)): # IMU 데이터만 filtering 하기
        raw_Data = datum.iloc[:,i]
        norm_Data = signal.resample_poly(raw_Data,101,len(raw_Data),padtype='line') # 101개로 모든 데이터 수 일치시키기
        norm_datum[datum.columns[i]] = norm_Data
    return norm_datum

In [5]:
# 파일 경로 설정
dataDir =     r'Z:\PROJECTS\iwalqq\Data\V3D\Output\IMU Deep Learning\Data\allnew_20220325_raw_byDeepak_csv\INC_ByStep\INC_ByZero\Included_checked'
rawDir =      os.path.join(dataDir, r'RAW')
filteredDir = os.path.join(dataDir, r'FILT')
nordDir =     os.path.join(dataDir,r'NORM')

# 절대로 rawDir는 여기서 만들면 안됨
# 이전 단계에서 미리 만들어져있어야 함
make_dir(nordDir)
dataExt = r".csv"
# 파일목록 가져오기
# 파일 목록 전체 들고오고 필요한 확장자만 고르고 들고온 파일 목록에서 .txt만 남기고 나머지 것들 제외시키기
dataList = natsorted([_ for _ in os.listdir(filteredDir) if _.endswith(dataExt)])

In [6]:
for file in tqdm(dataList):
    datum = pd.read_csv(os.path.join(filteredDir, file))
    # filt_datum = resample_ALL(datum)
    filt_datum = resample_poly_ALL(datum) # resample_ALL()보다 나은 결과를 보여주는듯?
    namefornorm = "N_"+ file
    filt_datum.to_csv(os.path.join(nordDir,namefornorm),index=False)

  0%|          | 0/876 [00:00<?, ?it/s]